In [ ]:
class FeatureSelector():
    
    def __init__(self, estimator, cv, metric, use_recursion):
        self.estimator = estimator
        self.cv = cv
        self.use_recursion = use_recursion
        self.metric = metric
        
    def fit(self, X, y):
        # валидируем каждый признак
        scores = []
        for i in tqdm_notebook(range(X.shape[1])):
            scores.append(cross_validate(self.estimator, X[:, i].reshape(-1,1),\
                                         y, scoring =self.metric, cv = self.cv)['test_score'].mean())
        # сохраняем 
        self.scores = scores
        
        # сортируем индивидуальные скоры в порядке убывания        
        order = np.argsort(scores)[::-1]        
        # лучший скор, лучшие признаки, признаки для проверки, счетчик ранней остановки
        best_score, best_features, to_drop, = .5, [], []
        # по признакам в порядке убывания индивидуальных скоров
        for i in tqdm_notebook(order):
            # добавляем признак в список лучших
            current_features = best_features+[i]
            # считаем валидацю
            mean_score = cross_validate(self.estimator, X[:, current_features],\
                                         y, scoring =self.metric, cv = self.cv)['test_score'].mean()
            # если есть улучшение
            if mean_score > best_score:
                # обновляем лучший скор
                best_score = mean_score
                # обновляем лучшие признаки
                best_features = current_features           

            # если улучшения нет
            else:
                # добавляем признак в кандидаты на удаление
                to_drop.append(i)
                
        if self.use_recursion:        
            # списки на удаление признаков ДО и ПОСЛЕ отбора
            to_drop_before = to_drop
            to_drop_after = []
            # запускаем бесконечный цикл
            while True:
                # добавляем последовательно признаки из кандидатов на удаление
                for i in tqdm_notebook(to_drop_before):
                    current_features = best_features+[i]
                    mean_score = cross_validate(self.estimator, X[:, current_features],\
                                             y, scoring =self.metric, cv = self.cv)['test_score'].mean()
                    # если есть улучшение скора
                    if mean_score > best_score:
                        # обновляем лучший скор
                        best_score = mean_score
                        # обновляем лучшие признаки
                        best_features = current_features            
                    else:
                        # добавляем признак в кандидаты на удаление ПОСЛЕ
                        to_drop_after.append(i)
                # если списки ДО и ПОСЛЕ отличаются меньше, чем на 5 признаков 
                if len(to_drop_before) == len(to_drop_after):
                    # останавливаем отбор
                    break            
                else:
                    # ДО --> ПОСЛЕ, ПОСЛЕ - пустой
                    to_drop_before = to_drop_after
                    to_drop_after = []
                    
        self.best_features = best_features            
        return self
    
    def transform(self, X):
        # оставляет в X отобранные признаки
        return X[:, self.best_features]
        
    def show_cv_weights(self):
        # выводит валидацию исходных признаков
        return self.scores
    
    def show_best_features(self):
        # выводит список отобранных признаков
        return self.best_features